In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
def convert(timestamp):
    year = int(timestamp[0:4])
    month = int(timestamp[5:7])
    day = int(timestamp[8:10])
    hour = int(timestamp[11:13])
    minute = int(timestamp[14:16])
    try:
        second = int(timestamp[17:19])
    except:
        second = 0
    time = datetime(year, month, day, hour, minute, second)
    return time

In [ ]:
df_simple_response_times = pd.read_csv("C:/Users/erikm/Documents/Studie/Master/ambulance-optimization/output/simulation/simple_response_times_fastest.csv")
df_simple_response_times["response_time"] = df_simple_response_times["response_time"]/60
times = [convert(x) for x in df_simple_response_times["timestamp"].values]

# Convert timestamp column to datetime
df_simple_response_times["timestamp"] = pd.to_datetime(df_simple_response_times["timestamp"])

# Sort the DataFrame based on timestamp
df_simple_response_times = df_simple_response_times.sort_values(by="timestamp")

In [ ]:
df_incidents = pd.read_csv("C:/Users/erikm/Documents/Studie/Master/ambulance-optimization/src/main/resources/data/incidents.csv")

start = datetime(2017, 8, 7, 0, 0, 0)
end = datetime(2017, 8, 14, 0, 0, 0)

rows = []
times2 = []
for incident in df_incidents.values:
    try:
        time = convert(incident[0])
        if start < time < end and incident[3] not in ["V1", "V2", "V"]:
            if time not in times:
                continue
            arrival = convert(incident[7])
            dispatch = convert(incident[6])
            delta = arrival - time
            dispatch_time = dispatch - time
            if delta.seconds >= 0:
                times2.append(time)
                response = {"time": time, "response_time": delta.seconds/60, "dispatch_time": dispatch_time.seconds/60}
                rows.append(response)
    except:
        continue

response_times = pd.DataFrame(rows)    

In [ ]:
df_simple_response_times = df_simple_response_times[df_simple_response_times["timestamp"].isin(times2)]

In [ ]:
cut_start = 0
cut_end = len(df_simple_response_times)
response_times_plot = response_times[cut_start:cut_end]
df_simple_response_times_plot = df_simple_response_times[cut_start:cut_end]

print(response_times_plot.shape)
print(df_simple_response_times_plot.shape)

In [ ]:
from scipy.stats import spearmanr, kendalltau

In [ ]:
print(response_times_plot["response_time"].corr(df_simple_response_times_plot["response_time"]))
print(spearmanr(response_times_plot["response_time"], df_simple_response_times_plot["response_time"]))
print(kendalltau(response_times_plot["response_time"], df_simple_response_times_plot["response_time"]))


In [ ]:
plt.rcParams["figure.figsize"] = (15,5)
plt.plot(response_times_plot["time"], response_times_plot["response_time"])
plt.plot(response_times_plot["time"], df_simple_response_times_plot["response_time"])
plt.gcf().autofmt_xdate()
plt.ylim(0, 100)

uses simulated dispatch time. 